<a href="https://colab.research.google.com/github/safal25/cv/blob/master/Regression_auto_mp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras

In [ ]:
dataset_path=keras.utils.get_file("auto-mpg.data","http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")

32768/30286 [================================] - 0s 5us/step


In [ ]:
dataset_path

'/root/.keras/datasets/auto-mpg.data'

In [ ]:
coloumn_names=["MPG","Cylinders","Displacement","Horsepower","Weight","Accelaration","Model Year","Origin"]

In [ ]:
import pandas as pd

In [ ]:
raw_data=pd.read_csv(dataset_path,names=coloumn_names,na_values='?',comment='\t',sep=" ",skipinitialspace=True)

In [ ]:
dataset=raw_data.copy()
dataset.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Accelaration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [ ]:
import seaborn as sns

In [ ]:
dataset.isna().sum()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Accelaration    0
Model Year      0
Origin          0
dtype: int64

In [ ]:
dataset=dataset.dropna()
dataset.isna().sum()

MPG             0
Cylinders       0
Displacement    0
Horsepower      0
Weight          0
Accelaration    0
Model Year      0
Origin          0
dtype: int64

In [ ]:
origin=dataset.pop('Origin')

In [ ]:
dataset['USA']=(origin==1)*1.0
dataset['Europe']=(origin==2)*1.0
dataset['Japan']=(origin==3)*1.0
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Accelaration,Model Year,USA,Europe,Japan
393,27.0,4,140.0,86.0,2790.0,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130.0,24.6,82,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295.0,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625.0,18.6,82,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720.0,19.4,82,1.0,0.0,0.0


In [ ]:
train_data=dataset.sample(frac=0.8,random_state=0)
test_data=dataset.drop(train_data.index)

In [ ]:
train_labels=train_data.pop('MPG')
test_labels=test_data.pop('MPG')

In [ ]:
train_stats=train_data.describe()
train_stats=train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Accelaration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


In [ ]:
def norm(x):
  return (x-train_stats['mean'])/train_stats['std']

norm_train_data=norm(train_data)
norm_test_data=norm(test_data)

In [ ]:
norm_train_data.head()

,Cylinders,Displacement,Horsepower,Weight,Accelaration,Model Year,USA,Europe,Japan
146,-0.869348,-1.009459,-0.784052,-1.025303,-0.379759,-0.516397,0.774676,-0.465148,-0.495225
282,-0.869348,-0.530218,-0.442811,-0.118796,0.624102,0.843910,0.774676,-0.465148,-0.495225
69,1.483887,1.482595,1.447140,1.736877,-0.738281,-1.060519,0.774676,-0.465148,-0.495225
378,-0.869348,-0.865687,-1.099044,-1.025303,-0.308055,1.660094,0.774676,-0.465148,-0.495225
331,-0.869348,-0.942365,-0.994047,-1.001603,0.875068,1.115971,-1.286751,-0.465148,2.012852


In [ ]:
model=keras.Sequential([keras.layers.Dense(64,activation='relu',input_shape=[len(train_data.keys())]),
                        keras.layers.Dense(64,activation='relu'),
                        keras.layers.Dense(1)
])

In [ ]:
opt=keras.optimizers.RMSprop(0.001)
model.compile(loss='mse',optimizer=opt,metrics=['mae','mse'])

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self,epochs,logs):
    if epochs%100==0: print(' ')
    print('.',end='')

early_stop=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)
history=model.fit(norm_train_data,train_labels,epochs=1000,validation_split=0.2,verbose=0,callbacks=[early_stop,PrintDot()])

 
...........................................................

In [ ]:
prediction_lables=model.predict(norm_test_data)

In [ ]:
import numpy as np

In [ ]:
test=np.array(test_labels)

In [ ]:
for i in range(25):
  print(prediction_lables[i],test[i])

[16.20423] 15.0
[10.952108] 10.0
[12.046513] 9.0
[23.408577] 25.0
[22.131319] 19.0
[13.450018] 14.0
[13.511989] 14.0
[13.219342] 13.0
[20.208128] 18.0
[29.974257] 35.0
[25.033276] 25.0
[25.000978] 19.0
[14.593717] 13.0
[26.519758] 28.0
[12.17323] 13.0
[13.8485775] 14.0
[14.207056] 15.0
[13.022407] 13.0
[18.250944] 18.0
[12.309328] 12.0
[12.273876] 16.0
[26.150705] 24.0
[19.707579] 19.0
[20.708681] 24.0
[30.014685] 31.0
